# Unión de tablas de actores con datos de Netflix
•	Objetivo:
Unir las colecciones de 'actores' y 'data_netflix' a través del id

In [ ]:
!pip install pymongo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 21.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import pymongo

In [ ]:
# Datos en formato CSV
data_netflix = pd.read_csv("data_netflix.csv")
mejores_shows = pd.read_csv("Mejores_shows_netflix.csv")
mejores_peliculas = pd.read_csv("Mejores_películas_Netflix.csv")
actores = pd.read_csv("Actores.csv")

In [ ]:
# Conexión a MongoDB
client = pymongo.MongoClient("mongodb://localhost:27017/")

# Acceso a la BD
db = client["netflix"]

# Acceso a la colección
collection = db['data_netflix_clean']

In [ ]:
pipeline = [
   {
    #Unir los datos de la colección 'actors' usando el campo 'Id' de 'netflix'
    $lookup: {
      from: "actor",                # Colección de actores
      localField: "id",             #Campo en 'netflix'
      foreignField: "id",            # Campo en 'actors'
      as: "actors_list"              # Resultado de la unión
    }
  },
   {
        $unwind: '$actors_list'
    },
    {
        "$replaceRoot": {
            "newRoot": {
                id: "$id",
								title: "$title",
								type: "$type",
								release_year: "$release_year",
								genres: '$genres',
								production_countries: "$production_countries",
								actor_name: '$actors_list.name',
                actor_role: '$actors_list.role'
            }
        }
    },
		{
		$out: 'cruce_data_netflix_actores'
		}
]

# Ejecutar la agregación
result = collection.aggregate(pipeline)
